In [78]:
import requests
from datetime import datetime
import pandas as pd
from ordered_set import OrderedSet
import time

### Get list of transactions Hashes

In [3]:
with open('ethplorer-api-keys.txt') as f:
    lines = f.readlines()
    
ethplorer_api_keys = lines[0].rstrip()
ethscan_api_keys = lines[1]

In [15]:
r_transactions = requests.get("https://api.etherscan.io/api?module=account&action=txlist&address=0x11111112542D85B3EF69AE05771c2dCCff4fAa26&startblock=12262083&endblock=12281283&sort=dsc&apikey={}".format(ethscan_api_keys))

In [136]:
lst_of_hashes = []
transactions = r_transactions.json()
for i in range(1000): ### Use 100 first len(transactions['result'])
    lst_of_hashes.append(transactions['result'][i]['hash'])

### Helper Functions

In [133]:
def create_output_dict():
    output_dict = {}
    output_dict['Volume_USD'] = []
    output_dict['Pairs'] = []
    output_dict['datetime'] = []
    output_dict['tx_hash'] = []
    
    return output_dict

In [44]:
def get_datetime(json_file):
    
    timestamp = json_file['timestamp']
    dt_object = datetime.fromtimestamp(timestamp)
    
    return dt_object

In [91]:
def get_transfer(json_file):
    
    transfers = []
    operations_list = json_file['operations']
    for i in operations_list:
        if i['type'] == 'transfer':

            ## Ignore CHI token
            if i['tokenInfo']['symbol'] != 'CHI':

                new_dict = {}
                decimals = int(i['tokenInfo']['decimals'])
                new_dict['no_of_tokens'] = int(i['value'])/10**decimals
                new_dict['name_of_token'] = i['tokenInfo']['symbol']
                new_dict['priority'] = int(i['priority'])

                if i['tokenInfo']['price'] == False:
                    new_dict['price_in_usd'] = 'no_price'
                else:
                    new_dict['price_in_usd'] = i['tokenInfo']['price']['rate']
                #new_dict['datetime'] = dt_object
                transfers.append(new_dict)    
    
    return transfers

In [89]:
def get_volUSD(transfers):
    priority_lst = []
    #token_lst = Orderedset()
    
    for idx in transfers:
        priority_lst.append(idx['priority'])
        #token_lst.add(idx['name_of_token'])

    index_min = min(range(len(priority_lst)), key=priority_lst.__getitem__)
    
    if transfers[index_min]['price_in_usd'] == 'no_price':
        Vol_USD = 0
        for idx in range(len(transfers)):
            if transfers[idx]['price_in_usd'] != 'no_price':
                Vol_USD = transfers[idx]['no_of_tokens'] * transfers[idx]['price_in_usd']
            if Vol_USD != 0:
                break
    
    else:
        Vol_USD = transfers[index_min]['no_of_tokens'] * transfers[index_min]['price_in_usd']
    return Vol_USD

In [119]:
def Sort_Tuple(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1])) 

def get_pair(transfers):
    
    
    priority_lst = []
    token_lst = OrderedSet()
    priority_dict = {}
    
    for idx in transfers:
        priority_lst.append(idx['priority'])
        token_lst.add(idx['name_of_token'])
        
    token_lst = list(token_lst)
    
    
    if len(token_lst) == 2:
        pairing = sorted(token_lst)[0] + "/"  + sorted(token_lst)[1]

    elif len(token_lst) == len(priority_lst):

        index_min = min(range(len(priority_lst)), key=priority_lst.__getitem__)
        low = token_lst[index_min]
        index_max = max(range(len(priority_lst)), key=priority_lst.__getitem__)
        high = token_lst[index_max]
        token_pair = [low,high]
        sorted_token_lst = sorted(token_pair)
        pairing = sorted_token_lst[0] + "/"  + sorted_token_lst[1]
    
    else:
        pairing = token_lst[0] +  "/" + token_lst[-1]
    return pairing

In [147]:
def get_pair(transfers):
    priority_tuple = []
    for idx in transfers:
        priority_tuple.append((idx['name_of_token'],idx['priority']))


    sorted_tuples = Sort_Tuple(priority_tuple)
    
    if sorted_tuples[0][0] != sorted_tuples[-1][0]:

        pairing = sorted_tuples[0][0] +  "/" + sorted_tuples[-1][0]
    
    else:
        pairing = "not_tracked"
    
    return pairing

In [130]:
def output_dict_append(output_dict):
    output_dict['datetime'].append(dt_object)
    output_dict['Pairs'].append(pairing)
    output_dict['Volume_USD'].append(volUSD)
    output_dict['tx_hash'].append(tx_hash)
    

In [79]:
import time

In [86]:
tx_hash

'0xfb7b930d61aac30e06bb0d8aaf05185960660e9af4a038649e3ab88560884602'

### Get Transactions Hashes

In [158]:
lst_of_hashes = []
start_block = 12282136
end_block = start_block + 500
counter = 0
while start_block >= (12282136 - 6400*3):


    r_transactions = requests.get('https://api.etherscan.io/api?module=account&action=txlist&address=0x11111112542D85B3EF69AE05771c2dCCff4fAa26&startblock={}&endblock={}&page=1&offset=0&sort=desc&apikey={}'.format(start_block,end_block,ethscan_api_keys))
    jsonfile = r_transactions.json()['result'] 
    for item in jsonfile:
        lst_of_hashes.append(item['hash'])
    
    start_block -= 500
    end_block = start_block + 500
    counter += 1
    
    if counter % 5 == 0:
        time.sleep(1)

### Main Function

In [173]:
output_dict = create_output_dict()

counter = 0
lst_of_hashes_test =['0x2c40c0ea0d96e4b200f8e7fbd77aa8a9a0e7b1c860cde05aa0e6c6015a2df26b']
for txh in lst_of_hashes:
    counter += 1
    tx_hash = txh
    r = requests.get('https://api.ethplorer.io/getTxInfo/{}?apiKey={}'.format(tx_hash,ethplorer_api_keys))
    json_file = r.json()
    
    if json_file['success'] == False:
        continue
        
    try:
        var = json_file['operations']
    except KeyError:
        continue
        
    ### Get Datetime
    dt_object = get_datetime(json_file)
    
    ### Get Transfers
    transfers = get_transfer(json_file)
    
    if len(transfers) == 0:
        continue
    ### Get priority
    
    volUSD = get_volUSD(transfers)
    
    
    ### Get pair
    
    pairing = get_pair(transfers)

    output_dict_append(output_dict)
    
    if counter % 10 == 0:
        time.sleep(0.01)

In [178]:
df = pd.DataFrame.from_dict(output_dict) 

In [179]:
df['date'] = df['datetime'].dt.date

In [180]:
sample = df[['date','Pairs','Volume_USD']].groupby(['date','Pairs']).sum()

### Total Daily Volume for all pairs in USD

In [191]:
sample

Volume_USD
date       Pairs                    
2021-04-18 1INCH/ARCX   2.452111e+04
           1INCH/COMP   4.163312e+04
           1INCH/DAI    1.248293e+04
           1INCH/USDC   3.137729e+05
           1INCH/USDT   1.709982e+05
...                              ...
2021-04-21 cBSN/USDC    8.793480e+04
           not_tracked  1.540345e+07
           ⚗️/UNI       1.546270e+03
           ⚗️/USDT      4.222126e+03
           ⚗️/WETH      2.554247e+02

[5077 rows x 1 columns]

### Total Daily Volume in USD

In [190]:
df[['date','Volume_USD']].groupby(['date']).sum()

,Volume_USD
date,
2021-04-18,6.606013e+07
2021-04-19,2.345165e+08
2021-04-20,2.096032e+08
2021-04-21,1.241235e+08
